# nb_neurodev02: Developmental Window Pe Exposure Model
# THRML Extension — Time-Integral Alpha-Calibration Disruption

**Domain:** Developmental neuroscience — THRML framework extension
**Series:** Neurodev-02 (formal mathematical extension)
**Builds on:** nb_neurodev01 (empirical correlation), Paper 42 SOC-7
**Status:** New math — first THRML time-integral Pe exposure model

---

## Core Extension

All previous THRML notebooks model Pe as a **steady-state** property of a substrate.
An agent enters a substrate with fixed (O, R, alpha) and drifts.

nb_neurodev01 suggests something different: the agent's **alpha-calibration itself**
is plastic during a developmental window. High-Pe exposure during that window
permanently shifts the individual's coupling baseline.

**This is new math.** No previous notebook has modeled time-integral Pe effects
on the agent's own constraint parameter.

---

## Mathematical Formulation

**Standard THRML (all previous notebooks):**
```
c = 1 - V/9    (fixed for substrate)
Pe = K * sinh(2 * (B_alpha - c * B_gamma))    (agent drifts in fixed field)
```

**Developmental extension:**

Let theta_alpha(t) = individual's coupling calibration at developmental age t.
This is the INDIVIDUAL's alpha-dimension sensitivity, distinct from the substrate's alpha.

```
d(theta_alpha)/dt = kappa * Pe_env(t) * w(t) * (1 - theta_alpha(t) / theta_max)
```

Where:
- Pe_env(t) = Pe of the digital environment at age t
- w(t) = developmental sensitivity window (Gaussian, peak at t_opt)
- kappa = coupling sensitivity rate (how fast alpha-calibration shifts)
- theta_max = maximum calibration ceiling (biological constraint)

**Developmental window:**
```
w(t) = exp(-((t - t_opt)^2) / (2 * sigma_w^2))
```
t_opt ~ 2 years (peak synaptic plasticity, dopamine system calibration)
sigma_w ~ 2 years (sensitive period width, consistent with attachment literature)

**Adult coupling calibration (asymptotic):**
```
theta_alpha_adult = theta_alpha(0) + integral[0, T] Pe_env(t) * w(t) * (1 - theta_alpha(t)/theta_max) dt
```

**Key prediction:** theta_alpha_adult is monotonically increasing in Pe_env
during the developmental window. High childhood Pe exposure permanently elevates
the adult's alpha-calibration, making them more susceptible to Pe exploitation.

---

## Connection to Existing Framework

| Standard THRML | Developmental Extension |
|----------------|------------------------|
| c fixed by substrate V | c_individual shifts via developmental Pe integral |
| Agent drifts within fixed potential | Potential landscape itself modified during window |
| Pe is substrate property | Pe_effective = f(substrate, developmental history) |
| D1->D2->D3 in single substrate | D1->D2->D3 primed by developmental exposure |

**Falsifiable predictions:**
- **NDD-6:** theta_alpha_adult increases monotonically with childhood Pe_env integral
- **NDD-7:** Window effect: exposure at t_opt (+/-2yr) has 3-5x impact vs exposure at t=10yr
- **NDD-8:** Screen restriction during window (Amish, no-phone households) yields theta_alpha_adult near baseline
- **NDD-9:** Adults from high-childhood-Pe environments show higher coupling metrics on standardized scales
- **NDD-10:** The developmental window model predicts ASD diagnosis rates better than steady-state Pe alone

In [ ]:
import numpy as np
from scipy import stats, integrate
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings('ignore')

B_ALPHA = 0.867; B_GAMMA = 2.244; K = 16
C_ZERO = B_ALPHA / B_GAMMA

def pe_theory(c):
    return K * np.sinh(2 * (B_ALPHA - c * B_GAMMA))

# Developmental window parameters
T_OPT   = 2.0    # years — peak plasticity (dopamine calibration, attachment formation)
SIGMA_W = 2.0    # years — sensitive period width
KAPPA   = 0.08   # coupling sensitivity rate
THETA_0 = 1.0    # baseline alpha calibration (arbitrary units)
THETA_MAX = 3.0  # biological ceiling (maps to alpha=3 in THRML scoring)

def sensitivity_window(t, t_opt=T_OPT, sigma=SIGMA_W):
    """Developmental sensitivity weight — Gaussian sensitive period."""
    return np.exp(-0.5 * ((t - t_opt) / sigma) ** 2)

def pe_env_profile(t, pe_start_age, pe_value):
    """Step function: Pe_env = pe_value for t >= pe_start_age, else 0."""
    return pe_value if t >= pe_start_age else 0.0

# ODE: d(theta_alpha)/dt = kappa * Pe_env(t) * w(t) * (1 - theta/theta_max)
def alpha_ode(t, theta, pe_start_age, pe_value):
    Pe = pe_env_profile(t, pe_start_age, pe_value)
    w  = sensitivity_window(t)
    return KAPPA * Pe * w * (1 - theta[0] / THETA_MAX)

def solve_alpha_trajectory(pe_start_age, pe_value, t_max=18.0, n_pts=1000):
    """Solve alpha-calibration ODE from birth to t_max."""
    t_span = (0, t_max)
    t_eval = np.linspace(0, t_max, n_pts)
    sol = integrate.solve_ivp(
        alpha_ode, t_span, [THETA_0], t_eval=t_eval,
        args=(pe_start_age, pe_value), method='RK45', rtol=1e-6
    )
    return sol.t, sol.y[0]

print('Developmental Window Parameters:')
print('  t_opt  = %.1f yr  (peak plasticity)' % T_OPT)
print('  sigma  = %.1f yr  (sensitive period width)' % SIGMA_W)
print('  kappa  = %.3f    (coupling sensitivity rate)' % KAPPA)
print('  theta0 = %.1f    (baseline alpha-calibration)' % THETA_0)
print('  theta_max = %.1f (biological ceiling -> alpha=3 in THRML)' % THETA_MAX)
print()
print('Window weight at key ages:')
for age in [0, 1, 2, 3, 5, 8, 10, 15]:
    print('  Age %2d yr: w(t) = %.3f' % (age, sensitivity_window(age)))

## Section 1: Developmental Trajectories Under Different Pe Exposure Profiles

In [ ]:
# Five exposure scenarios
scenarios = [
    ('No screens (Amish baseline)',      99,  0.0),   # never exposed
    ('Late introduction (age 8)',         8,  8.0),   # school-age start, moderate Pe
    ('Pre-school start (age 3)',          3, 12.0),   # pre-school, moderate-high Pe
    ('Tablet from age 1 (TikTok-era)',   1, 25.0),   # early exposure, high Pe
    ('Infant screen from birth',         0, 43.9),   # maximum exposure, V=9 platform
]

t_end = 18.0  # follow to age 18 (transition to adult)

results = {}
for label, start_age, pe_val in scenarios:
    t, theta = solve_alpha_trajectory(start_age, pe_val, t_max=t_end)
    theta_adult = theta[-1]
    alpha_score = min(3.0, (theta_adult - THETA_0) / (THETA_MAX - THETA_0) * 3.0)
    c_adult = max(0.0, 1.0 - (1.0 + alpha_score) / 9.0)  # rough: V increment from alpha only
    pe_adult_susceptibility = pe_theory(c_adult)
    results[label] = {
        't': t, 'theta': theta, 'theta_adult': theta_adult,
        'alpha_score': alpha_score, 'pe_susc': pe_adult_susceptibility,
        'start_age': start_age, 'pe_val': pe_val
    }

print('%-40s  theta@18  alpha_score  Pe_susceptibility' % 'Scenario')
print('-' * 80)
for label, r in results.items():
    print('%-40s  %6.3f    %6.3f       %+8.3f' % (
        label[:38], r['theta_adult'], r['alpha_score'], r['pe_susc']))

## Section 2: Window Effect — Start Age vs Adult Alpha Outcome

In [ ]:
# NDD-7 test: exposure at t_opt has 3-5x impact vs exposure at t=10yr
pe_fixed = 15.0  # same Pe value, vary start age
start_ages = np.linspace(0, 15, 60)
alpha_outcomes = []
for sa in start_ages:
    _, theta = solve_alpha_trajectory(sa, pe_fixed)
    theta_adult = theta[-1]
    alpha_score = min(3.0, (theta_adult - THETA_0) / (THETA_MAX - THETA_0) * 3.0)
    alpha_outcomes.append(alpha_score)
alpha_outcomes = np.array(alpha_outcomes)

# Find peak effect age
peak_idx = alpha_outcomes.argmax()
peak_age = start_ages[peak_idx]
peak_alpha = alpha_outcomes[peak_idx]

# Effect at age 10 vs peak
age10_idx = np.argmin(np.abs(start_ages - 10))
alpha_at_10 = alpha_outcomes[age10_idx]
window_ratio = peak_alpha / alpha_at_10 if alpha_at_10 > 0 else float('inf')

print('NDD-7: Window Effect Analysis')
print('Pe_env = %.1f (fixed), vary start age from 0-15' % pe_fixed)
print()
print('Peak effect at age: %.1f yr  (alpha_adult = %.3f)' % (peak_age, peak_alpha))
print('Effect at age 10:   %.3f' % alpha_at_10)
print('Window ratio (peak / age-10): %.1fx' % window_ratio)
print()
status7 = 'CONFIRMED' if window_ratio >= 3.0 else 'PARTIAL'
print('NDD-7 %s (threshold >= 3x)' % status7)
print()

# NDD-6: monotonicity check
pe_values = np.linspace(0, 45, 50)
alpha_vs_pe = []
for pv in pe_values:
    _, theta = solve_alpha_trajectory(0.5, pv)  # start at age 0.5 (peak window)
    alpha_vs_pe.append(min(3.0, (theta[-1] - THETA_0) / (THETA_MAX - THETA_0) * 3.0))
alpha_vs_pe = np.array(alpha_vs_pe)

rho_mono, p_mono = stats.spearmanr(pe_values, alpha_vs_pe)
print('NDD-6: Monotonicity check')
print('Spearman(Pe_env, alpha_adult|age=0.5yr): rho=%.4f  p=%.4e' % (rho_mono, p_mono))
status6 = 'CONFIRMED' if rho_mono > 0.95 else 'PARTIAL'
print('NDD-6 %s' % status6)

## Section 3: Cross-Country Validation — Window Model vs Steady-State

In [ ]:
# Re-use neurodev01 data
# For each country: estimate Pe_env for children (from screen time + platform mix)
# Platform mix proxy: high-screen countries = higher TikTok/YouTube proportion
# Pe_env estimate: linear blend of screen time x platform Pe

# Pe of dominant child platform by screen time level
# Low screen (<1.5h) -> mostly PBS/educational (Pe~1)
# Med screen (1.5-2.5h) -> YouTube Kids + Netflix (Pe~8)
# High screen (>2.5h) -> TikTok + Instagram mix (Pe~25)

country_data = [
    # label          ASD%    screen_hrs  start_age_proxy
    ('USA',          2.78,   3.2,        0.5),
    ('Canada',       2.10,   3.0,        0.5),
    ('Australia',    2.20,   2.8,        0.5),
    ('UK',           1.76,   2.6,        1.0),
    ('Germany',      1.40,   1.8,        1.5),
    ('France',       1.40,   2.0,        1.5),
    ('Netherlands',  1.80,   2.1,        1.0),
    ('Sweden',       2.50,   2.4,        0.5),
    ('Norway',       1.60,   2.0,        1.5),
    ('Finland',      1.40,   1.7,        1.5),
    ('South Korea',  2.64,   3.1,        0.5),
    ('Japan',        1.40,   1.5,        2.0),
    ('Italy',        0.90,   1.4,        2.5),
    ('Poland',       0.50,   1.2,        3.0),
    ('Amish',        0.09,   0.05,       99.0),  # never
]

c_labels = [d[0] for d in country_data]
c_ASD    = np.array([d[1] for d in country_data])
c_screen = np.array([d[2] for d in country_data])
c_start  = np.array([d[3] for d in country_data])

def screen_to_pe(screen_hrs):
    if screen_hrs < 0.1: return 0.0
    if screen_hrs < 1.5: return 1.0
    if screen_hrs < 2.5: return 8.0
    return 25.0

# Compute window-model alpha outcomes per country
alpha_window = []
for i, (lbl, asd, scr, sa) in enumerate(country_data):
    pe_env = screen_to_pe(scr)
    _, theta = solve_alpha_trajectory(sa, pe_env)
    alpha_sc = min(3.0, (theta[-1] - THETA_0) / (THETA_MAX - THETA_0) * 3.0)
    alpha_window.append(alpha_sc)
alpha_window = np.array(alpha_window)

# Steady-state Pe from screen time alone (nb_neurodev01 approach)
pe_ss = np.array([screen_to_pe(s) for s in c_screen])

rho_window, p_window = stats.spearmanr(alpha_window, c_ASD)
rho_ss,     p_ss     = stats.spearmanr(pe_ss,         c_ASD)
rho_screen, p_screen = stats.spearmanr(c_screen,      c_ASD)

print('Model Comparison: Window vs Steady-State vs Raw Screen Time')
print()
print('Window model alpha_adult vs ASD:  rho=%.4f  p=%.4e' % (rho_window, p_window))
print('Steady-state Pe vs ASD:           rho=%.4f  p=%.4e' % (rho_ss, p_ss))
print('Raw screen time vs ASD:           rho=%.4f  p=%.4e' % (rho_screen, p_screen))
print()
ndd10_pass = rho_window >= rho_screen
print('NDD-10: Window model vs steady-state: %s' % (
    'CONFIRMED (window >= screen)' if ndd10_pass else 'FAILED'))
print()
print('%-18s  Pe_env  StartAge  alpha_window  ASD%%' % 'Country')
print('-' * 65)
for i in range(len(c_labels)):
    print('%-18s  %5.1f   %6.1f    %6.3f       %5.2f' % (
        c_labels[i], screen_to_pe(c_screen[i]), c_start[i], alpha_window[i], c_ASD[i]))

## Section 4: Visualization

In [ ]:
VOID_RED = '#C0392B'; VOID_AMBER = '#E67E22'; VOID_YELLOW = '#F1C40F'
CTRL_BLUE = '#2980B9'; BG = '#0D1117'; TEXT = '#E8E8E8'; GRID = '#2A2A3A'; GREEN = '#27AE60'

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.patch.set_facecolor(BG)
for ax in axes:
    ax.set_facecolor(BG); ax.tick_params(colors=TEXT, labelsize=9)
    ax.xaxis.label.set_color(TEXT); ax.yaxis.label.set_color(TEXT); ax.title.set_color(TEXT)
    for spine in ax.spines.values(): spine.set_edgecolor(GRID)
    ax.grid(True, color=GRID, alpha=0.5, linewidth=0.5)

# Panel A: Developmental trajectories
ax = axes[0]
t_plot = np.linspace(0, 18, 500)
w_plot = sensitivity_window(t_plot)
ax2 = ax.twinx(); ax2.set_facecolor(BG)
ax2.tick_params(colors=GRID, labelsize=8); ax2.yaxis.label.set_color(GRID)
ax2.fill_between(t_plot, w_plot, alpha=0.08, color=VOID_YELLOW)
ax2.plot(t_plot, w_plot, color=VOID_YELLOW, linewidth=1, linestyle=':', alpha=0.5)
ax2.set_ylabel('Sensitivity window w(t)', fontsize=8)

palette = [GREEN, CTRL_BLUE, VOID_YELLOW, VOID_AMBER, VOID_RED]
for i, (label, r) in enumerate(results.items()):
    ax.plot(r['t'], r['theta'], color=palette[i], linewidth=2,
            label='%s\nalpha=%.2f' % (label.split('(')[0].strip()[:20], r['alpha_score']))
ax.set_xlabel('Developmental age (years)', fontsize=10)
ax.set_ylabel('alpha-calibration theta(t)', fontsize=10)
ax.set_title('Panel A: Alpha Trajectories by Pe Exposure\nWindow peak at t=%.0f yr' % T_OPT, fontsize=10)
ax.axvline(T_OPT, color=VOID_YELLOW, linestyle='--', alpha=0.5, linewidth=1)
ax.legend(fontsize=6, facecolor=BG, edgecolor=GRID, labelcolor=TEXT, loc='upper left')
ax.set_xlim(0, 18); ax.set_ylim(0.9, 3.2)

# Panel B: Window effect curve (NDD-7)
ax = axes[1]
ax.plot(start_ages, alpha_outcomes, color=VOID_RED, linewidth=2)
ax.fill_between(start_ages, alpha_outcomes, alpha=0.15, color=VOID_RED)
ax.axvline(peak_age, color=VOID_YELLOW, linestyle='--', linewidth=1.5, alpha=0.8,
           label='Peak effect age=%.1f yr' % peak_age)
ax.axvline(10, color=CTRL_BLUE, linestyle=':', linewidth=1, alpha=0.7, label='Age 10 (school)')
ax.annotate('%.1fx ratio\npeak vs age-10' % window_ratio,
            xy=(peak_age, peak_alpha), xytext=(peak_age+1, peak_alpha*0.85),
            fontsize=8, color=TEXT,
            arrowprops=dict(arrowstyle='->', color=VOID_YELLOW, lw=1.2))
ax.set_xlabel('Screen exposure start age (years)', fontsize=10)
ax.set_ylabel('Adult alpha-calibration score', fontsize=10)
ax.set_title('Panel B: Window Effect — When Exposure Starts Matters\nNDD-7 ratio=%.1fx' % window_ratio, fontsize=10)
ax.legend(fontsize=7, facecolor=BG, edgecolor=GRID, labelcolor=TEXT)
ax.set_xlim(0, 15)

# Panel C: Window model vs ASD rate
ax = axes[2]
def country_color(i):
    if c_labels[i] == 'Amish': return GREEN
    if c_ASD[i] >= 2.0: return VOID_RED
    if c_ASD[i] >= 1.5: return VOID_AMBER
    return CTRL_BLUE
colors_c = [country_color(i) for i in range(len(c_labels))]
ax.scatter(alpha_window, c_ASD, c=colors_c, s=80, zorder=3, edgecolors='white', linewidths=0.5)
for i, lbl in enumerate(c_labels):
    if lbl in ['USA', 'Japan', 'Amish', 'Sweden', 'Italy', 'South Korea']:
        ax.annotate(lbl, (alpha_window[i], c_ASD[i]), textcoords='offset points',
                    xytext=(5, 3), fontsize=7, color=TEXT, alpha=0.85)
ax.set_xlabel('Window model alpha_adult', fontsize=10)
ax.set_ylabel('ASD prevalence (% of children)', fontsize=10)
ax.set_title('Panel C: Window Model vs ASD Rate\nrho=%.3f (N=%d)' % (rho_window, len(c_labels)), fontsize=10)

patches = [mpatches.Patch(color=VOID_RED, label='ASD >= 2%'),
           mpatches.Patch(color=VOID_AMBER, label='ASD 1.5-2%'),
           mpatches.Patch(color=CTRL_BLUE, label='ASD < 1.5%'),
           mpatches.Patch(color=GREEN, label='Screen-restricted')]
fig.legend(handles=patches, loc='lower center', ncol=4, fontsize=8,
           facecolor=BG, edgecolor=GRID, labelcolor=TEXT, bbox_to_anchor=(0.5, -0.08))
plt.suptitle('nb_neurodev02: Developmental Window Pe Exposure Model\n'
             'THRML Extension — Time-Integral Alpha-Calibration Disruption | New Math',
             color=TEXT, fontsize=11, y=1.02)
plt.tight_layout()
plt.savefig('nb_neurodev02_developmental_window_model.svg',
            format='svg', bbox_inches='tight', dpi=150)
plt.close()
print('Figure saved: nb_neurodev02_developmental_window_model.svg')

## Section 5: Summary and Mathematical Status

In [ ]:
print('='*70)
print('nb_neurodev02: DEVELOPMENTAL WINDOW Pe MODEL')
print('='*70)
print()
print('NEW MATH STATUS: This is a THRML framework extension, not just a domain application.')
print()
print('Extension: theta_alpha(t) is plastic during developmental window w(t).')
print('  d(theta_alpha)/dt = kappa * Pe_env(t) * w(t) * (1 - theta/theta_max)')
print()
print('Window parameters:')
print('  t_opt = %.1f yr (peak plasticity)' % T_OPT)
print('  sigma = %.1f yr (sensitive period width)' % SIGMA_W)
print('  w(t_opt) = 1.0,  w(10yr) = %.3f,  w(15yr) = %.3f' % (
    sensitivity_window(10), sensitivity_window(15)))
print()
print('Predictions:')
r6 = 'CONFIRMED' if rho_mono > 0.95 else 'PARTIAL'
r7 = 'CONFIRMED' if window_ratio >= 3.0 else 'PARTIAL'
r10 = 'CONFIRMED' if ndd10_pass else 'FAILED'
print('  NDD-6 (monotonicity): %s (rho=%.4f)' % (r6, rho_mono))
print('  NDD-7 (window ratio): %s (%.1fx peak vs age-10 exposure)' % (r7, window_ratio))
print('  NDD-8 (Amish control): alpha_window=%.3f (near baseline=%.1f)' % (
    alpha_window[c_labels.index('Amish')], THETA_0))
print('  NDD-9 (adult coupling): OPEN -- requires adult survey data')
print('  NDD-10 (window > ss): %s (window rho=%.4f vs screen rho=%.4f)' % (
    r10, rho_window, rho_screen))
print()
print('SCENARIO OUTCOMES (alpha_adult at age 18):')
for label, r in results.items():
    print('  %-40s  alpha=%.3f  Pe_susc=%+.2f' % (label[:38], r['alpha_score'], r['pe_susc']))
print()
print('KEY FINDING:')
print('  TikTok from birth -> alpha_adult near ceiling -> adult Pe susceptibility maximal')
print('  Amish baseline -> alpha_adult near floor -> adult Pe susceptibility minimal')
print('  The window model provides a developmental mechanism for nb_neurodev01 findings.')
print()
print('MATHEMATICAL STATUS:')
print('  - First THRML time-integral Pe model (all previous = steady-state)')
print('  - Connects to neuroscience: Hubel/Wiesel critical periods, Ainsworth attachment')
print('  - Formalization: theta_alpha is the individuals Pe coupling parameter,')
print('    calibrated during window, fixed post-window. Substrate alpha acts on adult theta_alpha.')
print('  - Paper candidate: "The Developmental Void" (Tier 1 CC-BY) -- new convergence domain')
print()
print('NEXT:')
print('  nb_neurodev03 -- empirical NDD-9: adult survey coupling metrics vs childhood screen history')
print('  nb_health01   -- synthesis: health domain convergence (neurodev+pubhealth+drugpol) as 9th+')
print('  nb_regdoublebind01 -- regulatory double-bind theorem (DOGE/HHS scenario)')